<a href="https://colab.research.google.com/github/marina554/accounting-practice/blob/main/%E9%80%A3%E7%B5%90%E4%BC%9A%E8%A8%88%EF%BC%88Consolidated_Accounting%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =========================================
# 連結会計シンプルモデル（親1社・子1社）
# =========================================
import pandas as pd

# ---------------------------
# 1. 会社の単体財務データ
# ---------------------------
# 辞書形式で勘定科目と残高を定義
parent_accounts = {
    "現金": 1000,
    "売掛金": 500,
    "売上高": 2000,
    "仕入": 800,
    "親子間売掛金": 200  # 子会社との内部取引
}

child_accounts = {
    "現金": 300,
    "売掛金": 100,
    "売上高": 500,
    "仕入": 200,
    "親子間買掛金": 200  # 親会社との内部取引
}

# 持分比率（親会社が子会社株式を保有する割合）
ownership = 0.6  # 60%

# ---------------------------
# 2. 内部取引消去関数
# ---------------------------
def eliminate_internal_transaction(parent_accounts, child_accounts, parent_key, child_key):
    """
    親子間の内部取引を消去する関数
    parent_key: 親会社側の勘定科目
    child_key: 子会社側の勘定科目
    """
    # 消去する金額は両方の小さい方
    eliminated_amount = min(parent_accounts[parent_key], child_accounts[child_key])

    # 両方から消去
    parent_accounts[parent_key] -= eliminated_amount
    child_accounts[child_key] -= eliminated_amount

    return eliminated_amount

# 実際に内部取引を消去
eliminated = eliminate_internal_transaction(parent_accounts, child_accounts, "親子間売掛金", "親子間買掛金")
print(f"内部取引消去額: {eliminated}")

# ---------------------------
# 3. 連結財務諸表作成
# ---------------------------
def create_consolidated_bs(parent, child, ownership):
    """
    単純連結B/Sを作成
    parent, child: 単体財務データ（辞書）
    ownership: 親会社の持分比率
    """
    consolidated = {}
    for key in parent.keys():
        if key not in child:
            consolidated[key] = parent[key]
        else:
            # 単純合算（内部取引はすでに消去済み）
            consolidated[key] = parent[key] + child[key]
    # 非支配株主持分の計算（子会社残高の40%）
    non_controlling_interest = {key: child[key]*(1-ownership) for key in child.keys()}
    return consolidated, non_controlling_interest

# 連結B/S作成
consolidated_bs, nci = create_consolidated_bs(parent_accounts, child_accounts, ownership)

# DataFrameに変換して見やすく表示
df_consolidated = pd.DataFrame([consolidated_bs, nci], index=["連結", "非支配株主持分"])
print("\n連結B/S:")
print(df_consolidated)


内部取引消去額: 200

連結B/S:
             現金    売掛金     売上高      仕入  親子間売掛金  親子間買掛金
連結       1300.0  600.0  2500.0  1000.0     0.0     NaN
非支配株主持分   120.0   40.0   200.0    80.0     NaN     0.0
